In [2]:
# For CMAPS data pre-process understanding 

# %pip install torch

import os
import csv
import random
import numpy as np
# from numpy.core.fromnumeric import transpose
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy import interpolate
import math
import torch

import torch.utils.data as data
import logging


In [ ]:

class CMAPSS():
    def __init__(self, data_root, data_set, max_rul, seq_len):
        # load params
        self.data_root = data_root  # path to data directory
        self.data_set = data_set  # name of data set
        self.max_rul = max_rul   # max RUL value cap ?
        self.seq_len = seq_len
        self.column_names = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                             's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                             's15', 's16', 's17', 's18', 's19', 's20', 's21']
        self.mode = None
        self.val_fold = 0

        # load CMAPSS_data
        self.train_data_df, self.test_data_df, self.test_truth = self._get_data(data_root=data_root, data_set=data_set) 
        # helper function (get data)  to load raw data from txt files


        self.train_x, self.train_y, self.test_x, self.test_y = self._process(self.train_data_df, self.test_data_df, self.test_truth)
        print("\n\n\n\n")
        print(np.array(self.train_x).shape)
        print(np.array(self.test_x).shape)

        self.data_save()


    def _get_data(self, data_root, data_set):
        # helper function to load raw data from txt files from my local directory

        train_data_pt = os.path.join(data_root, 'CMAPSSData', 'train_' + data_set + '.txt')
        assert os.path.exists(train_data_pt), 'data path does not exist: {:}'.format(train_data_pt)
        # print(train_data_pt)
        test_data_pt = os.path.join(data_root, 'CMAPSSData', 'test_' + data_set + '.txt')
        assert os.path.exists(test_data_pt), 'data path does not exist: {:}'.format(test_data_pt)

        test_truth_pt = os.path.join(data_root, 'CMAPSSData', 'RUL_' + data_set + '.txt')
        assert os.path.exists(test_truth_pt), 'data path does not exist: {:}'.format(test_truth_pt)

        train_data_df = pd.read_csv(train_data_pt, sep=" ", header=None)
        train_data_df.drop(train_data_df.columns[[26, 27]], axis=1, inplace=True)
        train_data_df.columns = self.column_names
        train_data_df = train_data_df.sort_values(['id', 'cycle'])

        test_data_df = pd.read_csv(test_data_pt, sep=" ", header=None)
        test_data_df.drop(test_data_df.columns[[26, 27]], axis=1, inplace=True)
        test_data_df.columns = self.column_names
        test_data_df = test_data_df.sort_values(['id', 'cycle'])

        test_truth = pd.read_csv(test_truth_pt, sep=" ", header=None)
        test_truth.drop(test_truth.columns[[1]], axis=1, inplace=True)



        print("seeing the stucture of the orginal data")
        print("Training Data Summary:")
        print(train_data_df.info())

        rows, columns = train_data_df.shape
        print(f"Number of rows/ time steps: {rows}")
        print(f"Number of columns: {columns}")
        print(train_data_df.head(6))
        print("\n\n")

        print("\nTest Data Summary:")
        print(test_data_df.info())
   
        rows, columns = test_data_df.shape
        print(f"Number of rows/ time steps: {rows}")
        print(f"Number of columns: {columns}")
        print(test_data_df.head(6))

        print("\n\n")
        print("Test Truth Summary:")
        print(test_truth.info())
        print(test_truth.head(6))
        print("\n\n")
        

        print("\n\n" + "We can see how magnitude of difference sensor data can vary -> thats why we need to norlamize the data later")

  
        return train_data_df, test_data_df, test_truth
    


    

    def _process(self, train_df, test_df, test_truth):

        print("\n\n\n\n\nNow the processing data function")
        # process train data
        # to find the maximum cycle for each engine and merge it with the train data
        # as well as calculate the RUL at each time step
        train_rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index() # cycle there means time step 
        train_rul.columns = ['id', 'max']  # add column name

        print("train_rul" + " visualization\n")
        print("As we found already for each id the max cycle")
        print(train_rul.head(7))



        train_df = train_df.merge(train_rul, on=['id'], how='left')  # merge the max column to the train data
        # calculate the RUL at each time step
        print("\nAfter merging with the max cycle for corresponding id")
        print(train_df.head())
        print("\nWe now also have access to the actual RUL that it will decay for each machine, at their own timestep\n")

        print("\n\n")
        # drop the max as not needed anymore
        train_df.drop('max', axis=1, inplace=True)

        # drop some columns of sensor (  Some sensors may not be directly affected by degradation mode ???) maybe these sensor are not important factors to RUL
        train_df.drop(['s1', 's5', 's6', 's10', 's16', 's18', 's19'], axis=1, inplace=True)
        print("we remove sensors with constant values , specifically those with sensor indices 1,5,6,10,16,18,and 19.")
        # This dataset includes four subdata sets from FD001 to FD004,each collected under different operating conditions and faultmodes.

        print("\n\n\n\n\nNow used the max to cal the RUL at each time step")
        print("max is then removed from the data")
        print("We can now observe that some sensor data are removed")
        print(train_df.head(10))

  
        train_df['setting1'] = train_df['setting1'].round(1) # as its the only data with many decimal places




        # calculate the RUL at each time step
        train_y = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
        train_y.columns = ['id', 'max']
        train_y = train_df.merge(train_y, on=['id'], how='left')
        train_y['RUL'] = train_y['max'] - train_y['cycle']
        train_y = train_y[['id', 'RUL']]
        train_y = train_y.apply(lambda x: [y if y <= self.max_rul else self.max_rul for y in x])
        train_engine_num = train_df['id'].nunique()
        logging.info("CMPDataIter:: iterator initialized (train engine number: {:})".format(train_engine_num))  
        print("Normalising of training data")
        print("Get the max cycle for each engine and calculate the RUL at each time step, also cappingt the max rul at self defined maximal RUL")
        print("Using max - current cycle = RUL remaining at each time step")
        print("Therefore getting our y labels for training data")
        print(train_y.head(10))
        print("we can see that all RUL is 125, this is because it's RUL is capped at 125, the acutal RUL is actually more than 125")
        print("\n\n\n\n\n")




        print("Now processing the test data")
        # process test data
        # We dont know when the test engine will fail
        print("Test data summary before processing")
        print(test_df.info())
        print("\n\n")
        print("test truth summary before processing")
        print(test_truth.info())
        print(test_truth.head(10))


        # find the max cycle on the testing data
        test_rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
        test_rul.columns = ['id', 'max'] # get the max cycle/ time step -> last cycle time step


        # the test_truth is our ground truth for the test data
        # test truth more column is the RUL at each time step
        test_truth.columns = ['more'] # add column name
        test_truth['id'] = test_truth.index + 1  # adjusting the row index to start from 1 instead of 0
        test_truth['max'] = test_rul['max'] + test_truth['more']  # (predicted final cycle number) = max cycle + RUL at each time step to get the max RUL 
        # so max there is the actual max RUL of the engine at cycle 0, as they didint provide final cycle as test data
        #The testing file only contains the sensor measurements to certain running cycles for another certain number of engines

        # repeat the process
        test_truth.drop('more', axis=1, inplace= True)

        print("\n\n\nTest truth after processing")
        print(test_truth.head(10))
        print(test_truth.info())
        print("Now we know the max RUL and how much RUL is left at each time step")
        print("\n\n\n\n\n")


        test_df = test_df.merge(test_truth, on=['id'], how='left')
        test_y = pd.DataFrame(data=[test_df['max'] - test_df['cycle']]).T # now we have the RUL at each time step

        test_df.drop('max', axis=1, inplace=True)
        test_df.drop(['s1', 's5', 's6', 's10', 's16', 's18', 's19'], axis=1, inplace=True)

        test_df['setting1'] = test_df['setting1'].round(1)

        print("\n\n\nTest data after processing")
        print(test_df.head(10))

        print("\n\n\nTest y after processing")
        print(test_y.head(10))

        test_y = test_y.apply(lambda x: [y if y <= self.max_rul else self.max_rul for y in x])
        test_engine_num = test_df['id'].nunique()
        logging.info("CMPDataIter:: iterator initialized (test engine number: {:})".format(test_engine_num))






##############################################################################################################################
        # normailize both train and test data to setting 1
        # this selects the relevant feature columns (excluding id and cycle) from the dataset
        train_data = train_df.iloc[:, 2:]
        test_data = test_df.iloc[:, 2:]

        print("\n\n\n\nNormalising the data")
        print("bf")
        print("train data")
        print(train_data.head(10))
        print("\ntest data")
        print(test_data.head(10))

        # create a new dataframe to store the normalized data
        train_normalized = pd.DataFrame(columns=train_data.columns[3:])
        test_normalized = pd.DataFrame(columns=test_data.columns[3:])


        scaler = MinMaxScaler() # helps ensure that features with larger ranges don't dominate the model's training

        grouped_train = train_data.groupby('setting1')
        grouped_test = test_data.groupby('setting1')
        #  Grouping by setting1 allows you to normalize data specific to each operating condition, preserving the distribution of features within each group.

        print("\n\nGrouping by setting 1") # it iterate per ascending order of setting 1 index
        # print(grouped_train.head(6))

        # question: since setting 1 or 2 both exist, how can we just group it by setting 1 ??


        for train_idx, train in grouped_train: # group by setting1, so each train_idx might hv multiple train data rows

            print(train_idx) # to it will show train as a batch for its corresponding index 

            scaled_train = scaler.fit_transform(train.iloc[:, 3:]) # scale by each setting to make normalization consistent
            scaled_train_combine = pd.DataFrame(
                data=scaled_train,
                index=train.index,
                columns=train_data.columns[3:])
            train_normalized = pd.concat([train_normalized, scaled_train_combine])


            # after normalizing the training data, we normalize the test data too
            for test_idx, test in grouped_test: # for loop to find the right data setting
                # normalize the test data based on their setting
                if train_idx == test_idx:
                    scaled_test = scaler.transform(test.iloc[:, 3:])
                    scaled_test_combine = pd.DataFrame(
                        data=scaled_test,
                        index=test.index,
                        columns=test_data.columns[3:])
                    test_normalized = pd.concat([test_normalized, scaled_test_combine])



        train_normalized = train_normalized.sort_index()
        test_normalized = test_normalized.sort_index()
        print('train_normalized is '+ str(np.shape(train_normalized)))
        # diff@xuqing

        # normalize the setting data as well
        train_setting = scaler.fit_transform(train_df.iloc[:, 1:5])
        test_setting = scaler.transform(test_df.iloc[:, 1:5])



        train_setting = pd.DataFrame(
            data=train_setting,
            index=train_df.index,
            columns=train_df.columns[1:5])

        test_setting = pd.DataFrame(
            data=test_setting,
            index=test_df.index,
            columns=test_df.columns[1:5])
        
        print("\n\n\n\n After normalising the data")

        print("train setting")
        print(train_normalized.head(5))
        

        # normalize the target data as well
        print("\n\n\nnormalise target data so that it is between 0 and 1")
        train_y = train_y.apply(lambda x: (x / self.max_rul))  # norm_y = y/max_RUL
        test_y = test_y.apply(lambda x: (x / self.max_rul))  # norm_y = y/max_RUL
        
        
        print("\nshape of test_y")
        print(np.shape(test_y))


        condition_num = train_df['setting1'].nunique()

        if condition_num > 1:
            logging.info("CMPDataIter:: data includes multi operating conditions")
        else:
            logging.info("CMPDataIter:: data includes single operating conditions")


  

        # generate final data:
        # generate 9 x 15 windows to obtain train_x , 15 from the sliding window size
        seq_gen = []
        start_index = 0
        for i in range(train_engine_num):
            end_index = start_index + train_rul.loc[i, 'max']
            # print("\nCurrent end index: " + str(end_index))

            if end_index - start_index < self.seq_len - 1:
                print('train data less than seq_len!')
            # for sensor train matrix, number of 21 X 15 needed per data points (minus the first sequence length) per engine, so the array input start from start index
            val = list(self.gen_sequence(train_normalized.iloc[start_index:end_index, :], self.seq_len,
                                         train_normalized.columns))
            seq_gen.extend(val)
            start_index = end_index
        train_x = list(seq_gen)

        print("number of train engine num :" + str(train_engine_num))

        # generate 3 x 15 windows to obtain train_ops
        seq_gen = []
        start_index = 0
        for i in range(train_engine_num):
            end_index = start_index + train_rul.loc[i, 'max']
            # print(end_index)
            # for ops train matrix, number of 3 X 15 needed per data points (minus the first sequence length) per engine, so the array input start from start index
            # settings data are in the first 3 columns of Train_Norm
            val = list(
                self.gen_sequence(train_setting.iloc[start_index:end_index, :], self.seq_len, train_setting.columns))
            seq_gen.extend(val)
            start_index = end_index
        train_ops = list(seq_gen)


        print("\n\nShowing normalised training data")
        print(train_normalized.head(5))
        print("\n\nshowing train_x types of index 0")
        print(train_x[0])
        print("\n\nshowing train_x types of index 1")
        print(train_x[1])
        print("\n number of nested array in train_x")
        print(len(train_x))
        print("\n\ntrain_y types before processing")
        print(train_y.head(5))


        # generate train labels (y)
        seq_gen = []
        start_index = 0
        for i in range(train_engine_num):
            end_index = start_index + train_rul.loc[i, 'max']
            val = list(self.gen_labels(train_y.iloc[start_index:end_index, :], self.seq_len, train_y.columns))
            seq_gen.extend(val)
            start_index = end_index
        train_y = list(seq_gen)

        
        print("\n\nshowing train_y types")
        print(np.shape(train_y))
        print(str(train_y[0]))



        seq_gen = []
        start_index = 0
        for i in range(test_engine_num):
            end_index = start_index + test_rul.loc[i, 'max']
            # diff@xuqing
            # for test matrix, only 1 of n X 15 needed per engine, so the array input start from end index - sequence length
            if end_index - start_index < self.seq_len:
                # print('Sensor::test data ({:}) less than seq_len ({:})!'
                #       .format(end_index - start_index, self.seq_len))

                # simply pad the first data serveral times:
                # print('Sensor::Use first data to pad!')
                num_pad = self.seq_len - (end_index - start_index)
                new_sg = test_normalized.iloc[start_index:end_index, :]
                for idx in range(num_pad):
                    new_sg = pd.concat([new_sg.head(1), new_sg], axis=0)

                val = list(self.gen_sequence(new_sg, self.seq_len, test_normalized.columns))
            else:
                val = list(self.gen_sequence(test_normalized.iloc[end_index - self.seq_len:end_index, :], self.seq_len,
                                             test_normalized.columns))
            seq_gen.extend(val)
            start_index = end_index
        test_x = list(seq_gen)
        # print(np.shape(test_y))

        seq_gen = []
        start_index = 0
        for i in range(test_engine_num):
            end_index = start_index + test_rul.loc[i, 'max']
            # for test matrix, only 1 of n X 15 needed per engine, so the array input start from end index - sequence length
            # print(end_index - start_index)
            if end_index - start_index < self.seq_len:
                # print('Setting::test data ({:}) less than seq_len ({:})!'
                #       .format(end_index - start_index, self.seq_len))

                # simply pad the first data serveral times:
                # print('Setting::Use first data to pad!')
                num_pad = self.seq_len - (end_index - start_index)
                new_sg = test_setting.iloc[start_index:end_index, :]
                for idx in range(num_pad):
                    new_sg = pd.concat([new_sg.head(1), new_sg], axis=0)

                val = list(self.gen_sequence(new_sg, self.seq_len, test_setting.columns))
            else:
                val = list(self.gen_sequence(test_setting.iloc[end_index - self.seq_len:end_index, :], self.seq_len,
                                             test_setting.columns))

            seq_gen.extend(val)
            start_index = end_index
        test_ops = list(seq_gen)




        # print('label starts')
        seq_gen = []
        start_index = 0
        for i in range(test_engine_num):
            end_index = start_index + test_rul.loc[i, 'max']
            # print(end_index - start_index)
            # print(end_index - self.seq_len)
            if (end_index - self.seq_len) < 0:
                val = list([self.gen_test_labels(test_y.iloc[0:end_index, :], self.seq_len, test_y.columns)])
            else:
                val = list([self.gen_test_labels(test_y.iloc[end_index - self.seq_len:end_index, :], self.seq_len,
                                                 test_y.columns)])
            seq_gen.extend(val)
            start_index = end_index
        test_y = list(seq_gen)

        return train_x, train_y, test_x, test_y

    def gen_sequence(self, id_df, seq_length, seq_cols):

        # for one id I put all the rows in a single matrix
        data_matrix = id_df[seq_cols].values.astype(np.float32)
        num_elements = data_matrix.shape[0]
        # Iterate over two lists in parallel.
        # For example id1 (engine 1) have 192 rows and sequence_length is equal to 15
        # so zip iterate over two following list of numbers (0,177),(14,191)
        # 0 14 -> from row 0 to row 14
        # 1 15 -> from row 1 to row 15
        # 2 16 -> from row 2 to row 16
        # ...
        # 177 191 -> from row 177 to 191
        for start, stop in zip(range(0, num_elements - seq_length + 1), range(seq_length, num_elements + 1)):
            yield data_matrix[start:stop, :]

    def gen_labels(self, id_df, seq_length, label):
        # For example:
        # [[1]
        # [4]
        # [1]
        # [5]
        # [9]
        # ...
        # [200]]
        data_matrix = id_df[label].values
        num_elements = data_matrix.shape[0]
        label_matrix = []
        for i in range(num_elements - (seq_length - 1)):
            label_matrix.append(data_matrix[i + (seq_length - 1), :])

        return label_matrix

    # function to generate labels
    def gen_test_labels(self, id_df, seq_length, label):
        # For example:
        # [[1]]
        data_matrix = id_df[label].values
        num_elements = data_matrix.shape[0]
        # For the test labels, only 1 RUL is required per engine which is the last columns on each engine
        return data_matrix[-1, :]
    


    def data_save(self):
        if not os.path.exists('Processed_dataset'):
            os.mkdir('Processed_dataset')
        data_dir = os.path.join('Processed_dataset','CMAPSS')
        if not os.path.exists(data_dir):
            os.mkdir(data_dir)
        condition_data_dir = os.path.join(data_dir, f'{self.data_set}')
        if not os.path.exists(condition_data_dir):
            os.mkdir(condition_data_dir)

        torch.save({'samples':self.train_x,'labels':self.train_y,'max_ruls':self.max_rul},f'{condition_data_dir}/train.pt')
        torch.save({'samples':self.test_x,'labels':self.test_y,'max_ruls':self.max_rul},f'{condition_data_dir}/test.pt')


ROOT_PATH = os.path.join("C:\\Monash\\Research_Intern\\RUL_brenchmark\\Frank-Wang-oss-GNN_RUL_Benchmarking\\Data_Process", "Datasets")
data = CMAPSS(ROOT_PATH, data_set='FD004', max_rul=125, seq_len=50)


seeing the stucture of the orginal data
Training Data Summary:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 61249 entries, 0 to 61248
Data columns (total 26 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        61249 non-null  int64  
 1   cycle     61249 non-null  int64  
 2   setting1  61249 non-null  float64
 3   setting2  61249 non-null  float64
 4   setting3  61249 non-null  float64
 5   s1        61249 non-null  float64
 6   s2        61249 non-null  float64
 7   s3        61249 non-null  float64
 8   s4        61249 non-null  float64
 9   s5        61249 non-null  float64
 10  s6        61249 non-null  float64
 11  s7        61249 non-null  float64
 12  s8        61249 non-null  float64
 13  s9        61249 non-null  float64
 14  s10       61249 non-null  float64
 15  s11       61249 non-null  float64
 16  s12       61249 non-null  float64
 17  s13       61249 non-null  float64
 18  s14       61249 non-null  float64
 19  s15 